In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_csv('dt.csv')
df.head()

In [ ]:
df.dtypes

In [ ]:
df['dt'] = pd.Categorical(df['dt'])

In [ ]:
df['dt'] = df.dt.cat.codes


In [ ]:
df.head()

In [ ]:
target = df.pop('dt')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [ ]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
!pip install -q tensorflow-hub
import tensorflow_hub as hub

In [ ]:
train_examples_batch, train_labels_batch = next(iter(dataset.batch(10)))

In [ ]:
train_examples_batch


In [ ]:
train_labels_batch

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)


In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(dataset.shuffle(10000).batch(512),
                    epochs=10,
                    verbose=1)

In [ ]:
model = tf.keras.models.load_model('model.h5',custom_objects={'KerasLayer':hub.KerasLayer})